In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

In [2]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes                  edit: 2 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [3]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

 **Resources**

In [4]:
train_path = '../potato_and_catfood/train'
valid_path = '../potato_and_catfood/valid'
test_path = '../potato_and_catfood/test'

In [5]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(720,1280), classes=['potato', 'catfood'], batch_size = 10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(720,1280), classes=['potato', 'catfood'], batch_size = 4)
test_batches  = ImageDataGenerator().flow_from_directory(test_path, target_size=(720,1280), classes=['potato', 'catfood'], batch_size = 4)

Found 431 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [6]:
# train the model on the new data for a few epochs
model.fit_generator(train_batches, steps_per_epoch=20, epochs=10, validation_data=valid_batches, validation_steps=2)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

Epoch 1/10
20/20 [==============================] - 38s 2s/step - loss: 6.7408 - val_loss: 6.0443
Epoch 2/10
20/20 [==============================] - 27s 1s/step - loss: 8.2202 - val_loss: 6.0443
Epoch 3/10
20/20 [==============================] - 27s 1s/step - loss: 8.8650 - val_loss: 6.0443
Epoch 4/10
20/20 [==============================] - 28s 1s/step - loss: 8.3814 - val_loss: 6.0443
Epoch 5/10
20/20 [==============================] - 26s 1s/step - loss: 8.0771 - val_loss: 6.0443
Epoch 6/10
20/20 [==============================] - 26s 1s/step - loss: 7.7987 - val_loss: 6.0443
Epoch 7/10
20/20 [==============================] - 27s 1s/step - loss: 8.3814 - val_loss: 6.0443
Epoch 8/10
20/20 [==============================] - 26s 1s/step - loss: 8.7679 - val_loss: 6.0443
Epoch 9/10
20/20 [==============================] - 25s 1s/step - loss: 8.1217 - val_loss: 6.0443
Epoch 10/10
20/20 [==============================] - 27s 1s/step - loss: 8.2202 - val_loss: 6.0443


In [7]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

(0, 'input_1')
(1, 'conv2d')
(2, 'batch_normalization')
(3, 'activation')
(4, 'conv2d_1')
(5, 'batch_normalization_1')
(6, 'activation_1')
(7, 'conv2d_2')
(8, 'batch_normalization_2')
(9, 'activation_2')
(10, 'max_pooling2d')
(11, 'conv2d_3')
(12, 'batch_normalization_3')
(13, 'activation_3')
(14, 'conv2d_4')
(15, 'batch_normalization_4')
(16, 'activation_4')
(17, 'max_pooling2d_1')
(18, 'conv2d_8')
(19, 'batch_normalization_8')
(20, 'activation_8')
(21, 'conv2d_6')
(22, 'conv2d_9')
(23, 'batch_normalization_6')
(24, 'batch_normalization_9')
(25, 'activation_6')
(26, 'activation_9')
(27, 'average_pooling2d')
(28, 'conv2d_5')
(29, 'conv2d_7')
(30, 'conv2d_10')
(31, 'conv2d_11')
(32, 'batch_normalization_5')
(33, 'batch_normalization_7')
(34, 'batch_normalization_10')
(35, 'batch_normalization_11')
(36, 'activation_5')
(37, 'activation_7')
(38, 'activation_10')
(39, 'activation_11')
(40, 'mixed0')
(41, 'conv2d_15')
(42, 'batch_normalization_15')
(43, 'activation_15')
(44, 'conv2d_13')
(4

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)